# Data input/output

Chris Mutel ([PSI](https://www.psi.ch/)), Pascal Lesage ([CIRAIG](http://www.ciraig.org/en/))

*The following content is taken and adapted from [the course materials dispensed at the 2022 Brightcon conference](https://github.com/Depart-de-Sentier/teaching-material/tree/main/beginners/Projects%2C%20databases%2C%20exchanges%2C%20activities). If you need additional notebooks, go to this repository*.

## Learning objectives  
  - Learn how to input LCI data to Brightway in different ways:  
    - Programmatically, via dictionaries created directly in Python  
    - Excel/cvs importers  
    - SimaPro csv
      - Two unit-process example
      - Agribalyse
    - Importing ecoSpold, ecoSpold2

## Standard inputs and setup

In [1]:
import brightway2 as bw
import os
from pathlib import Path

In [2]:
bw.projects

Brightway2 projects manager with 16 objects:
	arezoo
	bw25
	coursePSI
	default
	ei38
	ei39
	new
	new4
	polyviz
	psi1
	strat_metals
	test
	testbio
	testbio2
	testbio3
	uvek
Use `projects.report()` to get a report on all projects.

Setting the project

In [3]:
bw.projects.set_current("coursePSI")

## Context

Performing LCA generally requires:
  - Background LCI data (e.g. an LCI database)  
  - Foreground LCI data  
  - Sets of characterization factors.    
This section will deal with the way data is input to Brightway

Useful documentation about what a database in Brightway is can be found [here](https://github.com/brightway-lca/brightway2/blob/master/notebooks/Databases.ipynb)
 and [here](https://2.docs.brightway.dev/intro.html#inventory-databases).

# Importing from CSV or Excel

We will import a dataset in Excel, but this dataset has errors that we will have to fix.

In [10]:
imp = bw.ExcelImporter(Path("..") / "files" / "lci-carbon-fiber.xlsx")
imp.apply_strategies()
imp.match_database(fields=('name', 'unit', 'location'))
imp.match_database("ecoinvent 3.9.1 cutoff", fields=('name', 'reference product', 'unit', 'location'))
imp.match_database("biosphere3", fields=('name', 'unit', 'categories'))
imp.statistics()

Extracted 1 worksheets in 0.03 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 2.68 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
10 datasets
109 exchanges
0 unlinked exchang

(10, 109, 0)

In [11]:
imp.write_database()

Writing activities to SQLite3 database:
0% [##########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/19/2024 11:52:24
  Finished: 03/19/2024 11:52:24
  Total time elapsed: 00:00:00
  CPU %: 108.70
  Memory %: 2.10
Created database: carbon fiber


Oops... we have some problems. Let's export our in-process data to a Excel sheet and see what went wrong. It is probably also sensible to load our input Excel sheet at the same time.

In [9]:
imp.write_excel()

Wrote matching file to:
/Users/romain/Library/Application Support/Brightway3/coursePSI.ad34cb91313c17c13fa891538321d355/output/db-matching-carbon-fiber.xlsx


It looks like we have a few small inconsistencies, like the name of the processes or their locations. Try to fix these on your own, and re-run the code two cells above to get an idea of your progress. The attributes used for matching (`fields`) must match exactly, though they are case-insensitive.

If you can't get it to work, you can change the name of the import file to "excel_importer_example_fixed.xlsx"; this already has the necessary corrections.

If you need to find the ecoinvent processes, you can search ecoinvent (as you should already know :)

In [ ]:
for u in imp.unlinked:
    print(u)

### Writing the database

Finally, we need to write this data to a new `Database`.

In [ ]:
imp.write_database()

In [ ]:
imp.metadata

In [12]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	carbon fiber
	ecoinvent 3.9.1 cutoff

Test to make sure we can do an LCA

In [13]:
carbon_fiber = [
    a for a in bw.Database("carbon fiber")
    if a["name"] == "carbon fiber production, weaved, at factory"
][0]

In [14]:
lca = bw.LCA(
    {carbon_fiber: 1}, 
    ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
)
lca.lci()
lca.lcia()
lca.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 3.58e+13)
  warnings.warn(msg, UmfpackWarning)


71.98903308507374

# Exporting databases

## To Excel

## As a bw2package file

In [ ]:
db = bw.Database("carbon fiber")

In [ ]:
db.export.